In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
Sequential = tf.keras.models.Sequential
Adam = tf.keras.optimizers.Adam
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("sensor_data.csv", parse_dates=["timestamp"]).set_index("timestamp")
df["delta"] = df.index.to_series().diff().dt.total_seconds()
time_steps = 1000

data = np.column_stack([df["delta"], df["temperature"], df["humidity"], df["pressure"], df["iaq"]])[1:]

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size, 1:])
    return np.array(X), np.array(y)

window_size = 12  # e.g., last 5*12 minutes
X, y = create_sequences(data_scaled, window_size)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = Sequential([
    Input(shape=(window_size, 5)),
    # unroll=True for tflite conversion
    LSTM(64, return_sequences=True, unroll=True),
    LSTM(32, unroll=True),
    Dense(4)  # predicting 4 variables
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mse"
)

model.summary()

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32
)

#pred_scaled = model.predict(X_test)

# Inverse scale temperature only
#temp_min = scaler.data_min_[0]
#temp_max = scaler.data_max_[0]
#pred_temp = pred_scaled[:, 0] * (temp_max - temp_min) + temp_min
#true_temp = y_test[:, 0] * (temp_max - temp_min) + temp_min



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 12, 64)         │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,468 (119.02 KB)

 Trainable params: 30,468 (119.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 0.1575 - val_loss: 0.0773
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0373 - val_loss: 0.0317
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0181 - val_loss: 0.0200
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0129 - val_loss: 0.0183
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0099 - val_loss: 0.0149
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0100 - val_loss: 0.0098
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0092 - val_loss: 0.0091
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0085 - val_loss: 0.0088
Epoch 9/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0080 - val_loss: 0.0112
Epoch 10/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0108 - val_loss: 0.0107
Epoch 11/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0091 - val_loss: 0.0082
Epoch 12/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0087 - val_l

In [15]:
print(scaler.data_min_)
print(scaler.data_max_)

[19.782961 14.92     33.38      1.005    50.      ]
[4.10227013e+03 3.40600000e+01 7.99300000e+01 1.01000000e+00
 2.15000000e+02]


In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.target_spec.supported_ops = [
#    tf.lite.OpsSet.TFLITE_BUILTINS,
#    tf.lite.OpsSet.SELECT_TF_OPS  # <-- IMPORTANT
#]
#converter._experimental_lower_tensor_list_ops = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpw30p_55j'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 12, 5), dtype=tf.float32, name='keras_tensor_20')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  135937804538448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804541904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804540560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804539984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804540752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804541328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804539408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135937804541712: TensorSpec(shape=(), dtype=tf.resource, name=None)
